In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的E2E ML: MLOps阶段5：部署：使用Vertex AI终结点和共享VM开始

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage5/get_started_with_vertex_endpoint_and_shared_vm.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage5/get_started_with_vertex_endpoint_and_shared_vm.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage5/get_started_with_vertex_endpoint_and_shared_vm.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示如何在Google Cloud上使用Vertex AI进行端到端MLOps生产操作。本教程涵盖第5阶段：部署：使用终端点和共享虚拟机开始托管模型。

### 目标

在本教程中，您将学习如何使用部署资源池来部署模型。部署资源池提供了在同一个（共享的）虚拟机上托管多个模型的能力。

部署资源池将模型部署组合在一起，共享虚拟机内的资源。在部署资源池内可以部署多个端点在同一个虚拟机上。每个端点可以拥有一个或多个部署的模型。给定端点的部署模型可以归为相同或不同的部署资源池。

本教程使用以下 Google Cloud ML 服务：

- `Vertex AI Training`
- `Vertex AI Model` 资源
- `Vertex AI Endpoint` 资源

执行的步骤包括：

- 上传一个预训练的图像分类模型作为 `Model` 资源（模型A）。
- 上传一个预训练的文本句子编码器模型作为 `Model` 资源（模型B）。
- 创建一个共享的虚拟机部署资源池。
- 列出共享的虚拟机部署资源池。
- 创建两个 `Endpoint` 资源。
- 使用部署资源池将第一个模型（模型A）部署到第一个 `Endpoint` 资源。
- 使用部署资源池将第二个模型（模型B）部署到第二个 `Endpoint` 资源。
- 使用第一个部署的模型（模型A）进行预测请求。
- 使用第二个部署的模型（模型B）进行预测请求。

### 模型

本教程使用的预训练模型来自[TensorFlow Hub](https://tfhub.dev/) 仓库：

- [图像分类](https://tfhub.dev/google/imagenet/inception_v3/classification/5): 使用ImageNet进行训练。
- [文本句子编码器](https://tfhub.dev/google/universal-sentence-encoder/4): 谷歌的通用句子编码器。

成本
本教程使用Google Cloud的计费组件：

- Vertex AI
- Cloud Storage

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

## 安装

安装执行这个笔记本所需要的软件包。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install the packages

! pip3 install --upgrade google-cloud-aiplatform \
                         tensorflow \
                         tensorflow-hub $USER_FLAG -q

重新启动内核

安装完额外的软件包后，您需要重新启动笔记本内核，以便它能够找到这些软件包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的谷歌云项目

**无论您使用什么笔记本环境，都需要执行以下步骤。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建账户时，您将获得300美元的免费信用额度，可用于支付计算/存储成本。

1. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用 Vertex AI、Compute Engine 和 Cloud Storage API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage_component)。

1. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保Cloud SDK在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行视为shell命令，并将以`$`为前缀的Python变量插入这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，这个变量用于整个笔记本的操作。下面是Vertex AI支持的区域。我们建议您选择离您最近的区域。

- 美洲: `us-central1`
- 欧洲: `europe-west4`
- 亚太: `asia-east1`

您可能不能使用多区域存储桶来训练Vertex AI。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果你正在进行实时教程会话，你可能正在使用共享的测试账户或项目。为了避免用户在创建的资源之间发生名称冲突，你需要为每个实例会话创建一个 UUID，并将其附加在你在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的Google Cloud帐户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经通过身份验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

1. **单击创建服务帐号**。

2. 在**服务帐号名称**字段中输入名称，然后单击**创建**。

3. 在**授予此服务帐号对项目的访问权限**部分，单击角色下拉列表。在过滤框中输入“Vertex AI”，并选择**Vertex AI管理员**。在过滤框中输入“Storage Object Admin”，并选择**Storage Object Admin**。

4. 单击创建。包含您密钥的JSON文件将下载到您的本地环境。

5. 在下面的单元格中将您的服务帐号密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是什么笔记本环境，以下步骤都是必须的。**

当您初始化用于 Python 的 Vertex SDK 时，您需要指定一个云存储的暂存桶。暂存桶是您的数据集和模型资源在会话间保留的地方。

请在下面设定您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中具有全局唯一性，包括您的组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶尚不存在时，才能运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查其内容来验证对您的云存储桶的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform
import google.cloud.aiplatform_v1beta1 as aip_beta
import tensorflow as tf
import tensorflow_hub as hub

### 初始化Python的Vertex AI SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

### Vertex AI常量

设置以下常量以用于Vertex AI：

- `API_ENDPOINT`：Vertex AI API服务端点。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}

设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量`DEPLOY_GPU/DEPLOY_NGPU`以使用支持 GPU 的容器映像，并指定分配给虚拟机（VM）实例的 GPU 数量。例如，要使用一个拥有 4 个 Nvidia Telsa K80 GPU 的容器映像分配给每个 VM，您可以指定：

    (aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定`(None, None)`来使用一个在 CPU 上运行的容器映像。

了解更多关于[您地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)的信息。

*注意*：TF 2.3 之前的 GPU 支持版本在此教程中无法加载自定义模型。这是一个已知问题，在 TF 2.3 中已修复。这是由于在服务功能中生成的静态图操作。如果您在自定义模型上遇到了此问题，请使用支持 GPU 的 TF 2.3 容器映像。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于预测的预构建Docker容器图像。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的VM的计算资源。
- `机器类型`
     - `n1-standard`：每个vCPU 3.75GB的内存。
     - `n1-highmem`：每个vCPU 6.5GB的内存
     - `n1-highcpu`：每个vCPU 0.9GB的内存
- `vCPUs`：\[2, 4, 8, 16, 32, 64, 96 \]个的数量

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

从 TensorFlow Hub 获取预训练模型

为了演示目的，本教程使用来自TensorFlow Hub（TFHub）的预训练模型，然后将其上传到“Vertex AI Model”资源。一旦有了“Vertex AI Model”资源，模型就可以部署到“Vertex AI Endpoint”资源。

### 下载预训练的图像分类模型

首先，从TensorFlow Hub下载预训练的图像分类模型。该模型以TF.Keras层的形式被下载。为了完成模型，例如，在这个例子中，您创建一个`Sequential()`模型，其中TFHub模型被指定为一层，并指定输入形状。下载的模型在ImageNet上进行了预训练。

In [ ]:
tfhub_model_icn = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/classification/5")]
)
tfhub_model_icn.build([None, 224, 224, 3])

### 保存模型工件

此时，模型位于内存中。接下来，您将模型工件保存到云存储位置。

In [ ]:
MODEL_ICN_DIR = BUCKET_URI + "/model_icn"
tfhub_model_icn.save(MODEL_ICN_DIR)

## 上传模型进行服务

接下来，您将上传您的TFHub图像分类模型到Vertex AI的`Model`服务，该服务将为您的模型创建一个Vertex AI的`Model`资源。在上传过程中，您需要定义一个serving函数来将数据转换为您的模型所期望的格式。如果您发送编码数据到Vertex AI，您的serving函数将确保数据在传递到模型之前在模型服务器上被解码。

### serving函数如何工作

当您向在线预测服务器发送请求时，该请求将由HTTP服务器接收。HTTP服务器从HTTP请求内容体中提取预测请求。提取的预测请求被转发到serving函数。对于Google预构建的预测容器，请求内容以`tf.string`的形式传递给serving函数。

serving函数由两部分组成：

- `预处理函数`：
  - 将输入（`tf.string`）转换为底层模型（动态图）期望的输入形状和数据类型。
  - 执行与训练底层模型期间相同的数据预处理 -- 例如，归一化，缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序预期的格式 -- 例如，压缩输出。
  - 打包输出供接收应用程序使用 -- 例如，添加标题，制作JSON对象等。

预处理和后处理函数均转换为与模型融合在一起的静态图。从底层模型输出传递给后处理函数。后处理函数将转换/打包后的输出传递回HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

在为TF.Keras模型构建serving函数时需要考虑的一个因素是它们作为静态图运行。这意味着您不能使用需要动态图的TF图操作。如果这样做，您将在编译serving函数时遇到错误，并指出您正在使用不受支持的EagerTensor。

### 图像数据的服务功能

### 预处理

为了将图像传递给预测服务，您需要将压缩（例如 JPEG）图像字节编码为 base 64 格式，以便在网络上传输二进制数据时使内容免受修改。由于部署的模型期望输入数据是原始（未压缩）字节，您需要确保将 base 64 编码的数据转换回原始字节，然后进行预处理以匹配模型的输入要求，然后将其作为输入传递给部署的模型。

为了解决这个问题，您需要定义一个服务函数（`serving_fn`）并将其附加到模型作为预处理步骤。添加一个 `@tf.function` 装饰器，使服务函数与底层模型融合在一起（而不是在 CPU 上游处理）。

当您发送预测或解释请求时，请求的内容被 base 64 解码为一个 Tensorflow 字符串（`tf.string`），然后传递给服务函数（`serving_fn`）。服务函数将 `tf.string` 预处理为原始（未压缩）的 numpy 字节（`preprocess_fn`）以满足模型的输入要求：

- `io.decode_jpeg` - 解压缩 JPG 图像，返回一个具有三个通道（RGB）的 Tensorflow 张量。
- `image.convert_image_dtype` - 将整数像素值更改为 32 位浮点数，并将像素数据重新缩放在 0 和 1 之间。
- `image.resize` - 将图像调整大小以匹配模型的输入形状。

此时，数据可以通过一个具体函数传递给模型（`m_call`）。服务函数是一个静态图，而模型是一个动态图。具体函数执行将输入数据从服务函数传递到模型，并将预测结果从模型传递回服务函数的任务。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(224, 224))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(tfhub_model_icn.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    tfhub_model_icn, MODEL_ICN_DIR, signatures={"serving_default": serving_fn}
)

获取服务函数签名

您可以通过重新加载模型到内存，并查询每个层对应的签名来获取您模型的输入和输出层的签名。

对于您的目的，您需要服务函数的签名。为什么？好吧，当我们将数据作为 HTTP 请求包发送进行预测时，图像数据是 base64 编码的，而我们的 TF.Keras 模型需要 numpy 输入。您的服务函数将执行将 base64 转换为 numpy 数组的操作。

在进行预测请求时，您需要将请求路由到服务函数，而不是模型，因此您需要知道服务函数的输入层名称 -- 这将在您之后进行预测请求时使用。

In [ ]:
loaded = tf.saved_model.load(MODEL_ICN_DIR)

serving_input_icn = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input_icn)

### 将TensorFlow Hub模型上传到`Vertex AI Model`资源

最后，您将TFHub模型和服务函数的模型工件上传到`Vertex AI Model`资源中。

*注意:* 当您将模型工件上传到`Vertex AI Model`资源时，您需要指定相应的部署容器镜像。

In [ ]:
model_icn = aiplatform.Model.upload(
    display_name="icn_" + UUID,
    artifact_uri=MODEL_ICN_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
)

print(model_icn)

### 下载预训练的句子编码器模型

接下来，您可以从TensorFlow Hub下载预训练的文本句子编码器模型。该模型将作为一个TF.Keras层进行下载。为了完成这个模型，在本例中，您可以使用下载的TFHub模型作为一个层来创建一个`Sequential()`模型，并指定输入形状给模型。

In [ ]:
tfhub_model_use = tf.keras.Sequential(
    [hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4")]
)

# force the model to build
tfhub_model_use.predict(["foo"])

### 保存模型工件

在这一点上，模型已经加载到内存中。接下来，您将模型工件保存到云存储位置。

In [ ]:
MODEL_USE_DIR = BUCKET_URI + "/model_use"
tfhub_model_use.save(MODEL_USE_DIR)

获取serving函数的签名

您可以通过重新加载模型到内存中，并查询每个层对应的签名来获取模型的输入和输出层的签名。

对于您的目的，您需要serving函数的签名。

在进行预测请求时，您需要将请求路由到serving函数而不是模型，因此您需要知道serving函数的输入层名称 -- 这将在您进行预测请求时使用。

In [ ]:
loaded = tf.saved_model.load(MODEL_USE_DIR)

serving_input_use = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input_use)

将TensorFlow Hub模型上传到 `Vertex AI Model` 资源中

最后，您将TFHub模型和服务功能中的模型构件上传到 `Vertex AI Model` 资源中。

*注意:* 当您将模型构件上传到 `Vertex AI Model` 资源时，您需指定相应的部署容器镜像。

In [ ]:
model_use = aiplatform.Model.upload(
    display_name="icn_" + UUID,
    artifact_uri=MODEL_USE_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
)

print(model_use)

## 创建部署资源池

目前，创建部署资源池只能通过基于REST的API（如CURL）和GAPIC API（Python）进行支持。

使用 `create_deployment_resource_pool` API 来创建一个资源池，具备以下配置：

- `dedicated_resources`：为共享虚拟机分配的计算（硬件）资源。
- `min_replica_count`：自动扩展，计算节点的最小数量。
- `max_replica_count`：自动扩展，计算节点的最大数量。

了解更多有关[部署资源池](https://googleapis.dev/python/aiplatform/latest/aiplatform_v1beta1/deployment_resource_pool_service.html)的信息。

In [ ]:
DEPLOYMENT_RESOURCE_POOL_ID = f"shared-vm-{UUID}"  # @param {type: "string"}
MIN_NODES = 1
MAX_NODES = 2

# Initialize request argument(s)
deployment_resource_pool = aip_beta.DeploymentResourcePool()
deployment_resource_pool.dedicated_resources.min_replica_count = MIN_NODES
deployment_resource_pool.dedicated_resources.max_replica_count = MAX_NODES
deployment_resource_pool.dedicated_resources.machine_spec.machine_type = DEPLOY_COMPUTE
if DEPLOY_NGPU:
    deployment_resource_pool.dedicated_resources.machine_spec.accelerator_type = DEPLOY_GPU
    deployment_resource_pool.dedicated_resources.machine_spec.accelerator_count = DEPLOY_NGPU

request = aip_beta.CreateDeploymentResourcePoolRequest(
    parent=f"projects/{PROJECT_ID}/locations/{REGION}",
    deployment_resource_pool=deployment_resource_pool,
    deployment_resource_pool_id=DEPLOYMENT_RESOURCE_POOL_ID,
)

pool_client = aip_beta.services.deployment_resource_pool_service.DeploymentResourcePoolServiceClient(
    client_options=client_options
)

op = pool_client.create_deployment_resource_pool(request=request)
print(op)

result = op.result()
print(result)

deployment_pool_id = result.name

## 获取部署资源池

使用`GetDeploymentResourcePool` API来查看您创建的部署资源池。

了解更多关于 [获取部署资源池](https://googleapis.dev/python/aiplatform/latest/aiplatform_v1beta1/deployment_resource_pool_service.html)。

In [ ]:
response = pool_client.get_deployment_resource_pool(name=deployment_pool_id)
print(response)

列出所有部署资源池

使用 `ListDeploymentResourcePools` API 列出所有部署资源池。

了解更多关于[列出部署资源池](https://googleapis.dev/python/aiplatform/latest/aiplatform_v1beta1/deployment_resource_pool_service.html)。

In [ ]:
pools = pool_client.list_deployment_resource_pools(
    parent=f"projects/{PROJECT_ID}/locations/{REGION}"
)
for pool in pools:
    print(pool)

## 创建两个`Endpoint`资源

接下来，您可以使用`Endpoint.create()`方法创建两个`Endpoint`资源。至少，您需要指定端点的显示名称。可选地，您也可以指定项目和位置（地区），否则这些设置将继承您使用`init()`方法初始化Vertex AI SDK时设置的值。

在本例中，指定了以下参数：

- `display_name`：`Endpoint`资源的可读名称。

此方法将返回一个`Endpoint`对象。

了解更多关于[Vertex AI端点](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
endpoint_icn = aiplatform.Endpoint.create(display_name="icn_" + UUID)

print(endpoint_icn)

endpoint_use = aiplatform.Endpoint.create(display_name="use_" + UUID)

print(endpoint_use)

## 在部署资源池中部署模型

在创建了一个模型和一个终端之后，您可以使用 DeployModel API 进行部署。在下面的 CURL 命令示例中，注意您是如何指定 DeployedModel 的 `shared_resources` 为创建的资源池的部署资源名称。

相同部署资源池的模型部署可以同时启动。

### 部署图像分类模型

接下来，您可以将图像分类模型部署到一个 `Endpoint` ，使用您的部署资源池。

In [ ]:
import json
import pprint

pp = pprint.PrettyPrinter(indent=4)


SHARED_RESOURCE = "projects/{project_id}/locations/{region}/deploymentResourcePools/{deployment_resource_pool_id}".format(
    project_id=PROJECT_ID,
    region=REGION,
    deployment_resource_pool_id=DEPLOYMENT_RESOURCE_POOL_ID,
)

DEPLOY_MODEL_PAYLOAD = {
    "deployedModel": {
        "model": model_icn.resource_name,
        "shared_resources": SHARED_RESOURCE,
    },
    "trafficSplit": {"0": 100},
}
DEPLOY_MODEL_REQUEST = json.dumps(DEPLOY_MODEL_PAYLOAD)
pp.pprint("DEPLOY_MODEL_REQUEST: " + DEPLOY_MODEL_REQUEST)

In [ ]:
ENDPOINT_ID = endpoint_icn.name

output = ! curl -X POST \
 -H "Authorization: Bearer $(gcloud auth print-access-token)" \
 -H "Content-Type: application/json" \
https://{REGION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}:deployModel \
-d '{DEPLOY_MODEL_REQUEST}'

for line in output:
    if '"name"' in line:
        operation_id = line.split(":")[-1].strip()[:-1]
        break
print(operation_id)

等待部署完成

接下来，您将查询操作的状态，等待操作状态'完成'设置为'true'。

In [ ]:
import time

done = False
while done != '"done": true':
    status = ! curl -X GET \
 -H "Authorization: Bearer $(gcloud auth print-access-token)" \
 -H "Content-Type: application/json" \
https://{REGION}-aiplatform.googleapis.com/v1beta1/{operation_id}
    for line in status:
        if '"done"' in line.strip():
            done = line.strip()[0:-1]
    print("DONE status:", done)
    time.sleep(30)

部署文本句子编码器模型

接下来，您将使用您的部署资源池将文本句子编码器模型部署到一个“终端节点”上。

In [ ]:
DEPLOY_MODEL_PAYLOAD = {
    "deployedModel": {
        "model": model_use.resource_name,
        "shared_resources": SHARED_RESOURCE,
    },
    "trafficSplit": {"0": 100},
}
DEPLOY_MODEL_REQUEST = json.dumps(DEPLOY_MODEL_PAYLOAD)
pp.pprint("DEPLOY_MODEL_REQUEST: " + DEPLOY_MODEL_REQUEST)

In [ ]:
ENDPOINT_ID = endpoint_use.name

output = ! curl -X POST \
 -H "Authorization: Bearer $(gcloud auth print-access-token)" \
 -H "Content-Type: application/json" \
https://{REGION}-aiplatform.googleapis.com/v1beta1/projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}:deployModel \
-d '{DEPLOY_MODEL_REQUEST}'

for line in output:
    if '"name"' in line:
        operation_id = line.split(":")[-1].strip()[:-1]
        break
print(operation_id)

等待部署完成

接下来，您将查询操作的状态，等待操作状态`完成`被设置为`true`。

In [ ]:
done = False
while done != '"done": true':
    status = ! curl -X GET \
 -H "Authorization: Bearer $(gcloud auth print-access-token)" \
 -H "Content-Type: application/json" \
https://{REGION}-aiplatform.googleapis.com/v1beta1/{operation_id}
    for line in status:
        if '"done"' in line.strip():
            done = line.strip()[0:-1]
    print("DONE status:", done)
    time.sleep(30)

获取端点的部署细节

列出在端点上部署的模型。

In [ ]:
print(endpoint_icn.list_models())
print(endpoint_use.list_models())

### 创建图像分类模型的测试示例

接下来，您要测试已部署的图像分类模型。首先，您需要为提供功能对测试数据进行编码，格式如下：

`{ serving_input: { 'b64': base64编码的字节 } }`

In [ ]:
! gsutil cp gs://cloud-ml-data/img/flower_photos/daisy/100080576_f52e8ee070_n.jpg test.jpg

import base64

with open("test.jpg", "rb") as f:
    data = f.read()
b64str = base64.b64encode(data).decode("utf-8")

### 为图像分类模型进行预测请求

最终，您会发起一个预测请求。由于该模型是在ImageNet上训练的，预测将返回相应1000个类别的概率。

In [ ]:
# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{serving_input_icn: {"b64": b64str}}]

prediction = endpoint_icn.predict(instances=instances)

print(prediction)

### 创建测试示例文本句编码器模型

接下来，您将测试已部署的文本句编码器模型。首先，您需要对用于服务功能的测试数据进行编码，其格式为：

`"word1 word2 ... wordN"`

In [ ]:
instance = "the brown fox jumped over the laxy dog"

### 为文本句子编码模型发起预测请求

最终，您发起一个预测请求。预测将返回一个包含500个元素的嵌入向量。

In [ ]:
endpoint_use.predict([instance])

解除部署模型

当您完成预测后，您需要从`Endpoint`资源中解除部署模型。这将取消所有计算资源并终止已部署模型的计费。

In [ ]:
endpoint_icn.undeploy_all()
endpoint_use.undeploy_all()

删除`Model`资源

`delete()`方法将删除模型。

In [ ]:
model_icn.delete()
model_use.delete()

删除`Endpoint`资源

方法'delete()'将删除该端点。

In [ ]:
endpoint_icn.delete()
endpoint_use.delete()

删除`DeploymentResourcePool`

方法 'delete_deployment_resource_pool()' 将删除您的部署资源池。

In [ ]:
response = pool_client.delete_deployment_resource_pool(name=deployment_pool_id)
print(response)

进行清理

要清理在此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

!rm -f test.jpg